## 당뇨병 여부 판단
- 예측컬럼: Outcome (0 정상, 1 당뇨) 당뇨병일 확률 예측
- 평가지표: roc-auc
- 제출파일명: result.csv (1개컬럼, 컬럼명 pred)

## 라이브러리 및 데이터 불러오기

In [73]:
# 라이브러리
import pandas as pd

In [74]:
# 데이터 불러오기
train = pd.read_csv("/kaggle/input/bigdatacertificationkr/diabetes_train.csv")
test = pd.read_csv("/kaggle/input/bigdatacertificationkr/diabetes_test.csv")
train.shape, test.shape

((576, 10), (192, 9))

## EDA

In [75]:
train.head()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,377,1,87,60,37,75,37.2,0.509,22,0
1,370,3,173,82,48,465,38.4,2.137,25,1
2,408,8,197,74,0,0,25.9,1.191,39,1
3,535,4,132,0,0,0,32.9,0.302,23,1
4,691,13,158,114,0,0,42.3,0.257,44,1


In [76]:
train['Outcome'].value_counts()

0    381
1    195
Name: Outcome, dtype: int64

In [78]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        576 non-null    int64  
 1   Pregnancies               576 non-null    int64  
 2   Glucose                   576 non-null    int64  
 3   BloodPressure             576 non-null    int64  
 4   SkinThickness             576 non-null    int64  
 5   Insulin                   576 non-null    int64  
 6   BMI                       576 non-null    float64
 7   DiabetesPedigreeFunction  576 non-null    float64
 8   Age                       576 non-null    int64  
 9   Outcome                   576 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 45.1 KB


In [79]:
train.isnull().sum()

id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [81]:
test.isnull().sum()

id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [82]:
train.describe()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000
mean,385.189236,3.866319,120.170139,68.421875,21.067708,82.133681,31.843403,0.479325,33.354167,0.338542
std,221.420334,3.388140,32.014163,20.080641,15.933359,115.444040,7.798820,0.343584,11.972995,0.473625
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,192.750000,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.244000,24.000000,0.000000
50%,385.500000,3.000000,116.000000,72.000000,23.000000,45.000000,32.050000,0.369000,29.000000,0.000000
75%,575.250000,6.000000,140.000000,80.000000,33.000000,130.000000,36.600000,0.637750,41.000000,1.000000
max,766.000000,17.000000,199.000000,114.000000,99.000000,846.000000,59.400000,2.420000,81.000000,1.000000


## 데이터 전처리

In [83]:
from sklearn.preprocessing import StandardScaler

cols = train.select_dtypes('number').columns[:-1]

scaler = StandardScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

train.describe()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,5.760000e+02,5.760000e+02,5.760000e+02,5.760000e+02,5.760000e+02,5.760000e+02,5.760000e+02,5.760000e+02,5.760000e+02,576.000000
mean,1.337665e-16,-1.746288e-16,9.478336e-17,-1.966020e-17,1.098658e-16,2.097088e-16,-6.693141e-16,-8.962738e-17,1.561251e-16,0.338542
std,1.000869e+00,1.000869e+00,1.000869e+00,1.000869e+00,1.000869e+00,1.000869e+00,1.000869e+00,1.000869e+00,1.000869e+00,0.473625
min,-1.741141e+00,-1.142125e+00,-3.756918e+00,-3.410317e+00,-1.323388e+00,-7.120772e-01,-4.086654e+00,-1.169069e+00,-1.032733e+00,0.000000
25%,-8.698682e-01,-8.467216e-01,-6.618489e-01,-3.200823e-01,-1.323388e+00,-7.120772e-01,-5.830820e-01,-6.855065e-01,-7.819512e-01,0.000000
50%,1.404722e-03,-2.559140e-01,-1.303724e-01,1.783427e-01,1.213787e-01,-3.219392e-01,2.651386e-02,-3.213784e-01,-3.639817e-01,0.000000
75%,8.591170e-01,6.302973e-01,6.199474e-01,5.770826e-01,7.495383e-01,4.149883e-01,6.104426e-01,4.614969e-01,6.391449e-01,1.000000
max,1.721349e+00,3.879739e+00,2.464484e+00,2.271727e+00,4.895391e+00,6.622518e+00,3.536503e+00,5.653235e+00,3.982900e+00,1.000000


In [36]:
train = train.drop('id', axis =1)
test = test.drop('id', axis =1)

In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               192 non-null    float64
 1   Glucose                   192 non-null    float64
 2   BloodPressure             192 non-null    float64
 3   SkinThickness             192 non-null    float64
 4   Insulin                   192 non-null    float64
 5   BMI                       192 non-null    float64
 6   DiabetesPedigreeFunction  192 non-null    float64
 7   Age                       192 non-null    float64
dtypes: float64(8)
memory usage: 12.1 KB


In [86]:
X_train = train.iloc[:,:-1]
y_train = train['Outcome'].map(lambda x: 1 if x>1 else 0)

X_train, y_train

(           id  Pregnancies   Glucose  BloodPressure  SkinThickness   Insulin  \
 0   -0.037017    -0.846722 -1.037009      -0.419767       1.000802 -0.061847   
 1   -0.068659    -0.255914  1.651637       0.676768       1.691778  3.319349   
 2    0.103110     1.221105  2.401957       0.278028      -1.323388 -0.712077   
 3    0.677178     0.039490  0.369841      -3.410317      -1.323388 -0.712077   
 4    1.382333     2.698124  1.182687       2.271727      -1.323388 -0.712077   
 ..        ...          ...       ...            ...            ...       ...   
 571  1.065917    -0.551318 -0.880692       0.377713      -0.067069 -0.712077   
 572 -1.162553    -0.846722 -0.099109       0.975823       0.184195  0.545034   
 573 -1.483488    -1.142125 -0.630586       0.975823       2.445569  0.241594   
 574 -0.199745    -0.846722 -0.786902       0.278028      -0.004253 -0.079187   
 575 -1.216795     0.334894  0.119734       0.278028      -1.323388 -0.712077   
 
           BMI  DiabetesPe

## 학습 및 예측

In [88]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## 파일 생성

In [124]:
predictions = rf.predict_proba(test)[:,0]
output = pd.DataFrame({'pred': predictions})
output

,pred
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
187,1.0
188,1.0
189,1.0
190,1.0


In [122]:
output.to_csv('result.csv', index=False)

In [ ]:
output = pd.DataFrame({'pred': pred})
output.head()

In [ ]:
# 수험번호.csv로 출력
output.to_csv('result.csv', index=False)